In [34]:
import os, shutil
import random
import tensorflow as tf
from tensorflow.keras import (models, layers, optimizers)

import plotly.express as ex
import plotly.graph_objects as go

In [35]:
data_path = "../data/processed"

In [36]:
print(len(os.listdir(data_path + "/train/dogs")))
print(len(os.listdir(data_path + "/train/cats")))

1414
1422


In [43]:
IMG_SIZE = 150

# https://www.tensorflow.org/guide/keras/preprocessing_layers
data_augmentation = tf.keras.Sequential([
    layers.Resizing(IMG_SIZE, IMG_SIZE),
    layers.Rescaling(1./255),
    layers.RandomRotation(.4),
    layers.RandomWidth(.2),
    layers.RandomHeight(.2),
    # layers.RandomCrop(.2, .2),
    layers.RandomZoom(.2),
    layers.RandomFlip("horizontal"),
])

In [44]:
model = models.Sequential([data_augmentation])
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.build(input_shape=(None, IMG_SIZE, IMG_SIZE, 3))
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_21 (Sequential)  (None, 150, 150, 3)       0         
                                                                 
 conv2d_40 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_42 (Conv2D)          (None, 34, 34, 128)     

In [45]:
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['acc']
)

In [46]:
batch_size = 16
x_train, x_valid = tf.keras.utils.image_dataset_from_directory(
    (data_path + "/train"),
    batch_size=batch_size,
    image_size=(IMG_SIZE, IMG_SIZE),
    validation_split=.2,
    subset="both",
    seed=42
)

Found 2836 files belonging to 2 classes.
Using 2269 files for training.
Using 567 files for validation.


In [47]:
x_train_count = len(x_train) * batch_size
x_valid_count = len(x_valid) * batch_size
print(x_train_count)
print(x_valid_count)

2272
576


In [48]:
history = model.fit(
    x_train.repeat(),
    steps_per_epoch=int(x_train_count/batch_size),
    epochs=30,
    validation_data=x_valid.repeat(),
    validation_steps=int(x_valid_count/batch_size)
)

Epoch 1/25
142/142 [==============================] - 28s 174ms/step - loss: 2.2403 - acc: 0.5280 - val_loss: 0.7933 - val_acc: 0.5203
Epoch 2/25
142/142 [==============================] - 24s 171ms/step - loss: 0.7558 - acc: 0.5108 - val_loss: 0.6993 - val_acc: 0.5450
Epoch 3/25
142/142 [==============================] - 24s 172ms/step - loss: 0.7386 - acc: 0.5346 - val_loss: 0.7005 - val_acc: 0.5450
Epoch 4/25
142/142 [==============================] - 24s 171ms/step - loss: 0.7172 - acc: 0.5584 - val_loss: 0.7107 - val_acc: 0.5220
Epoch 5/25
142/142 [==============================] - 25s 178ms/step - loss: 0.6959 - acc: 0.5518 - val_loss: 0.8293 - val_acc: 0.5062
Epoch 6/25
142/142 [==============================] - 25s 179ms/step - loss: 0.7014 - acc: 0.5641 - val_loss: 0.6955 - val_acc: 0.5485
Epoch 7/25
142/142 [==============================] - 25s 177ms/step - loss: 0.7019 - acc: 0.5800 - val_loss: 0.6845 - val_acc: 0.5802
Epoch 8/25
142/142 [==============================] - 2

In [49]:
# save model
model.save("./model.h5")

In [ ]:
# load model
# model = models.load_model("./model.h5")

In [50]:
acc = history.history['acc']
vall_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = [x for x in range(1, len(acc) + 1)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=epochs, y=acc, name="Training Accuracy"))
fig.add_trace(go.Scatter(x=epochs, y=vall_acc, name="Validation Accuracy"))
fig.update_layout(
    title="Training and Validation Accuracy",
    xaxis_title="Epochs",
    yaxis_title="Accuracy"
)
fig.show()

In [51]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=epochs, y=loss, name="Training Loss"))
fig.add_trace(go.Scatter(x=epochs, y=val_loss, name="Validation Loss"))
fig.update_layout(
    title="Training and Validation Loss",
    xaxis_title="Epochs",
    yaxis_title="Loss"
)
fig.show()